In [ ]:
# 라이브러리 임포트
from utils.hprint import hprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# 데이터 로드
path = "../resources/classification/10. Dataset_Pasteurizer/dataset/pasteurizer.csv"
df = pd.read_csv(path)

In [ ]:
# 4. 데이터 확인
hprint("데이터 확인:")
print(df.head())
hprint("데이터 정보:")
print(df.info())
hprint("데이터 통계 정보:")
print(df.describe())

In [ ]:
# 5. 데이터 전처리
df = df.dropna()  # 결측치 제거
df = df[(df['MIXA_PASTEUR_STATE'] <= 1) & (df['MIXB_PASTEUR_STATE'] <= 1)]  # 비정상 데이터 제거

In [ ]:
# 6. 독립변수와 종속변수 설정
X = df[['MIXA_PASTEUR_TEMP', 'MIXB_PASTEUR_TEMP']]
y = df['INSP'].apply(lambda x: 1 if x == 'OK' else 0)  # 양품(OK)을 1로, 불량(NG)을 0으로 변환

In [ ]:
# 7. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 8. 의사결정나무 모델 생성 및 학습
dt_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_clf.fit(X_train, y_train)

In [ ]:
# 9. 모델 예측 및 평가
y_pred = dt_clf.predict(X_test)
hprint("Accuracy:")
print(accuracy_score(y_test, y_pred))
hprint("Precision:")
print(precision_score(y_test, y_pred))
hprint("Recall:")
print(recall_score(y_test, y_pred))
hprint("F1 Score:")
print(f1_score(y_test, y_pred))

In [ ]:
# 10. 모델 시각화
plt.figure(figsize=(10, 7))  # 그래프 크기 조정
plot_tree(
    dt_clf,
    feature_names=['MIXA_PASTEUR_TEMP', 'MIXB_PASTEUR_TEMP'],
    class_names=['NG', 'OK'],
    filled=True,
    rounded=True
)
plt.show()

In [ ]:
# 11. 결과 해석
print(confusion_matrix(y_test, y_pred))  # 혼동 행렬 출력